In [3]:
import torch
import einops
import itertools

test_tensor1 = [
    [0, 1, 2, 2],
    [3, 4, 2, 2], 
    [5, 6, 7, 8], 
    [9, 10, 11, 12]
]

test_tensor2 = [
    [13, 14, 15, 16],
    [17, 18, 19, 20],
    [21, 21, 22, 22],
    [21, 21, 22, 22],
]

test_tensor3 = [
    [23, 24, 25, 26],
    [27, 28, 29, 30],
    [31, 32, 33, 34],
    [35, 36, 37, 38],
]

mask16 = [
    [[1, 1, 0, 0], 
    [1, 1, 0, 0], 
    [1, 1, 1, 1],
    [1, 1, 1, 1]],
    [[1, 1, 1, 1], 
    [1, 1, 1, 1], 
    [0, 0, 0, 0],
    [0, 0, 0, 0]],
    [[1, 1, 1, 1], 
    [1, 1, 1, 1], 
    [1, 1, 1, 1],
    [1, 1, 1, 1]],
]
mask32  = [
    [[0, 1], [0, 0]],
    [[0, 0], [1, 1]],
    [[0, 0], [0, 0]],
]

grp_token = torch.ones(1) * 69
cls_token = torch.ones(1) * 420
cls_token.unsqueeze(0).unsqueeze(0)

test_tensor1 = torch.tensor(test_tensor1).unsqueeze(0).unsqueeze(-1)
test_tensor2 = torch.tensor(test_tensor2).unsqueeze(0).unsqueeze(-1)
test_tensor3 = torch.tensor(test_tensor3).unsqueeze(0).unsqueeze(-1)
mask32 = torch.tensor(mask32)
mask16 = torch.tensor(mask16)


#test_tensor = torch.cat([test_tensor1, test_tensor2], dim=0)
test_tensor = torch.cat([test_tensor1, test_tensor2, test_tensor3], dim=0)
print("test_tensor shape: ", test_tensor.shape)
print("B: {}, C: {}, H: {}, W: {}".format(test_tensor.shape[0], test_tensor.shape[3], test_tensor.shape[1], test_tensor.shape[2]))

print("mask 16 shape: ", mask16.shape)
print("mask 32 shape: ", mask32.shape)

B = test_tensor.shape[0]
num_tokens = []
for idx in range(B):
    base = mask16[idx].sum().item() + mask32[idx].sum().item()
    #if self.cls_token is not None:
    base += 1 # handle cls token
    #if self.reg_token is not None:
    #    base += self.num_reg_tokens
    num_tokens.append(int(base))
#block_mask = BlockDiagonalMask.from_seqlens(num_tokens)

cls_token_indices = [0] + list(itertools.accumulate(num_tokens[:-1]))
print("Cls token indices: ", cls_token_indices)

B, H, W, C = test_tensor.shape
test_tensor = test_tensor.reshape(B, H*W, 1)
print("test_tensor shape: ", test_tensor.shape)
patchembed = einops.rearrange(test_tensor, 'b (h w) c -> b c h w', h=4, w=4)
patches_32 = einops.rearrange(patchembed, 'b c (h p1) (w p2)-> b c (h w) (p1 p2)', p1=2, p2=2)
patches_32 = patches_32.flatten(3)
patches_16 = patchembed.flatten(2)

expanded_token = grp_token.view(1, -1, 1, 1).expand(
    patches_32.shape[0],
    -1,
    patches_32.shape[2],
    1
)

patches_32 = torch.cat([expanded_token, patches_32], dim=-1)
patches_32 = patches_32.flatten(2)
print("Patches 32 shape: " , patches_32.shape)
patches_to_mask = torch.cat([patches_16, patches_32], dim=2).permute(0, 2, 1)
mask16 = mask16.flatten(1)
mask32 = mask32.flatten(1)

seqlens_32 = torch.ones_like(mask32, device=mask32.device, dtype=torch.int32) * 5
seqlens_16 = torch.ones_like(mask16, device=mask16.device, dtype=torch.int32)
combined_seqlens = torch.cat([seqlens_16, seqlens_32], dim=1)
seq_mask = torch.cat([mask16, mask32], dim=1).bool()
all_seqlens = [int(x) for x in combined_seqlens[seq_mask].tolist()]

mask32 = mask32.unsqueeze(-1).repeat(1, 1, 4)
mask32_extract = mask32 * 0
# Add mask for group token; I think this is wrong ??
# TODO: Check that it's correct, i just re-ordered.
mask32 = torch.cat([mask32[:, :, :1], mask32], dim=-1)
mask32 = mask32.flatten(1)
print("Mask32 after adding class token: ", mask32.shape)
print(mask32)

class_token_mask = torch.ones_like(mask32_extract[:, :, :1])
mask32_extract = torch.cat([class_token_mask, mask32_extract], dim=-1)
mask32_extract = mask32_extract.flatten(1)
print("Mask 32 extract: ")
print(mask32_extract)
# # Could replace with multiplication...
mask32_extract = torch.logical_and(mask32_extract, mask32)
print("mask32_extract shape: ", mask32_extract.shape)
print("mask32 extract: ")
print(mask32_extract)

combined_mask = torch.cat([mask16, mask32], dim=1).bool()
combined_extract_mask = torch.cat([mask16, mask32_extract], dim=1).bool()
extract_mask = combined_extract_mask[combined_mask]
full_patches = patches_to_mask[combined_mask].unsqueeze(0)
# PRETENDE WE DO THE ATTENTION HERE>
output = full_patches[0][extract_mask]

expanded_output = torch.zeros((output.shape[0] + B, output.shape[1]), device=output.device)
expanded_output_mask = torch.ones((output.shape[0] + B, 1), dtype=torch.bool, device=output.device)
cls_token_indices_tensor = torch.tensor(cls_token_indices, device=expanded_output.device)
expanded_output[cls_token_indices] = cls_token.reshape(1, 1)
# expanded_output[cls_token_indices] = cls_token.squeeze(0).expand(len(cls_token_indices), -1)
expanded_output_mask[cls_token_indices] = False

non_cls_indices = torch.where(expanded_output_mask)[0]
expanded_output[non_cls_indices] = output
expanded_output= expanded_output.unsqueeze(0)

print(expanded_output)

test_tensor shape:  torch.Size([3, 4, 4, 1])
B: 3, C: 1, H: 4, W: 4
mask 16 shape:  torch.Size([3, 4, 4])
mask 32 shape:  torch.Size([3, 2, 2])
Cls token indices:  [0, 14, 25]
test_tensor shape:  torch.Size([3, 16, 1])
Patches 32 shape:  torch.Size([3, 1, 20])
Mask32 after adding class token:  torch.Size([3, 20])
tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
Mask 32 extract: 
tensor([[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]])
mask32_extract shape:  torch.Size([3, 20])
mask32 extract: 
tensor([[False, False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [False, False, Fal

In [19]:
test_tensor1 = [
    [0, 1, 2, 2],
    [3, 4, 2, 2], 
    [5, 6, 7, 8], 
    [9, 10, 11, 12]
]

test_tensor2 = [
    [13, 14, 15, 16],
    [17, 18, 19, 20],
    [21, 21, 22, 22],
    [21, 21, 22, 22],
]

test_tensor3 = [
    [23, 24, 25, 26],
    [27, 28, 29, 30],
    [31, 32, 33, 34],
    [35, 36, 37, 38],
]

mask16 = [
    [[1, 1, 0, 0], 
    [1, 1, 0, 0], 
    [1, 1, 1, 1],
    [1, 1, 1, 1]],
    [[1, 1, 1, 1], 
    [1, 1, 1, 1], 
    [0, 0, 0, 0],
    [0, 0, 0, 0]],
    [[1, 1, 1, 1], 
    [1, 1, 1, 1], 
    [1, 1, 1, 1],
    [1, 1, 1, 1]],
]
mask32  = [
    [[0, 1], [0, 0]],
    [[0, 0], [1, 1]],
    [[0, 0], [0, 0]],
]

grp_token = torch.ones(1) * 69
cls_token = torch.ones(1) * 420

test_tensor1 = torch.tensor(test_tensor1).unsqueeze(0).unsqueeze(-1)
test_tensor2 = torch.tensor(test_tensor2).unsqueeze(0).unsqueeze(-1)
test_tensor3 = torch.tensor(test_tensor3).unsqueeze(0).unsqueeze(-1)
mask32 = torch.tensor(mask32)
mask16 = torch.tensor(mask16)
#test_tensor = torch.cat([test_tensor1, test_tensor2], dim=0)
test_tensor = torch.cat([test_tensor1, test_tensor2, test_tensor3], dim=0)

B = test_tensor.shape[0]
num_tokens = []
for idx in range(B):
    base = mask16[idx].sum().item() + mask32[idx].sum().item()
    #if self.cls_token is not None:
    base += 1 # handle cls token
    #if self.reg_token is not None:
    #    base += self.num_reg_tokens
    num_tokens.append(int(base))
cls_token_indices = [0] + list(itertools.accumulate(num_tokens[:-1]))
print("Cls token indices: ", cls_token_indices)

num_elements = (mask32.sum() + mask16.sum()).int().item()
B, H, W, C = test_tensor.shape
test_tensor = test_tensor.reshape(B, H*W, 1)
print(test_tensor.shape)
to_cat = [cls_token.expand(test_tensor.shape[0], 1, 1)]
print(to_cat[0].shape)
patchembed = torch.cat(to_cat + [test_tensor], dim=1)

cls_toks = patchembed[:, 0]
patchembed = patchembed[:, 1:]
patchembed = einops.rearrange(patchembed, 'b (h w) c -> b c h w', h=4, w=4)

mask16 = mask16.flatten(1)
num_32_patches = mask32.sum().int()
patch32_seqlens = [5 for _ in range(num_32_patches.item())]
mask32 = mask32 * 2
mask32 = mask32.flatten(1)

combined_mask = torch.cat([mask16, mask32], dim=1)
clstok_locs = torch.ones_like(mask16[:, :1])*-1
combined_mask = torch.cat([clstok_locs, combined_mask], dim=1)

valid_patches = combined_mask[combined_mask != 0]
grptok_locs = torch.where(valid_patches == 2)
mask16_locs = torch.where(valid_patches == 1)

# expanded_token = grp_token.view(1, -1, 1, 1).expand(
#     patches_32.shape[0],
#     1
# )

expanded_output = torch.zeros((num_elements + B, C), device=test_tensor.device, dtype=test_tensor.dtype)
expanded_output[grptok_locs] = 69
expanded_output[mask16_locs] = test_tensor[mask16.bool()]
expanded_output[cls_token_indices] = cls_toks.long()
expanded_output = expanded_output.unsqueeze(0)
print(cls_token_indices)
expanded_output


Cls token indices:  [0, 14, 25]
torch.Size([3, 16, 1])
torch.Size([3, 1, 1])
[0, 14, 25]


tensor([[[420],
         [  0],
         [  1],
         [  3],
         [  4],
         [  5],
         [  6],
         [  7],
         [  8],
         [  9],
         [ 10],
         [ 11],
         [ 12],
         [ 69],
         [420],
         [ 13],
         [ 14],
         [ 15],
         [ 16],
         [ 17],
         [ 18],
         [ 19],
         [ 20],
         [ 69],
         [ 69],
         [420],
         [ 23],
         [ 24],
         [ 25],
         [ 26],
         [ 27],
         [ 28],
         [ 29],
         [ 30],
         [ 31],
         [ 32],
         [ 33],
         [ 34],
         [ 35],
         [ 36],
         [ 37],
         [ 38]]])

In [23]:
import torch.nn.functional as F


test_tensor1 = [
    [0, 1, 2, 2],
    [3, 4, 2, 2], 
    [5, 6, 7, 8], 
    [9, 10, 11, 12]
]

test_tensor2 = [
    [13, 14, 15, 16],
    [17, 18, 19, 20],
    [21, 21, 22, 22],
    [21, 21, 22, 22],
]

test_tensor3 = [
    [23, 24, 25, 26],
    [27, 28, 29, 30],
    [31, 32, 33, 34],
    [35, 36, 37, 38],
]

mask16 = [
    [[1, 1, 0, 0], 
    [1, 1, 0, 0], 
    [1, 1, 1, 1],
    [1, 1, 1, 1]],
    [[1, 1, 1, 1], 
    [1, 1, 1, 1], 
    [0, 0, 0, 0],
    [0, 0, 0, 0]],
    [[1, 1, 1, 1], 
    [1, 1, 1, 1], 
    [1, 1, 1, 1],
    [1, 1, 1, 1]],
]
mask32  = [
    [[0, 1], [0, 0]],
    [[0, 0], [1, 1]],
    [[0, 0], [0, 0]],
]

grp_token = torch.ones(1) * 69
cls_token = torch.ones(1) * 420

test_tensor1 = torch.tensor(test_tensor1).unsqueeze(0).unsqueeze(-1)
test_tensor2 = torch.tensor(test_tensor2).unsqueeze(0).unsqueeze(-1)
test_tensor3 = torch.tensor(test_tensor3).unsqueeze(0).unsqueeze(-1)
mask32 = torch.tensor(mask32)
mask16 = torch.tensor(mask16)
#test_tensor = torch.cat([test_tensor1, test_tensor2], dim=0)
test_tensor = torch.cat([test_tensor1, test_tensor2, test_tensor3], dim=0)

B = test_tensor.shape[0]
num_tokens = []
for idx in range(B):
    base = mask16[idx].sum().item() + mask32[idx].sum().item()
    #if self.cls_token is not None:
    base += 1 # handle cls token
    #if self.reg_token is not None:
    #    base += self.num_reg_tokens
    num_tokens.append(int(base))
cls_token_indices = [0] + list(itertools.accumulate(num_tokens[:-1]))
print("Cls token indices: ", cls_token_indices)

pos_embeds = torch.ones((1, 17)) * 0.1
pos_embeds[:, 0] = -0.1
pos_embeds32 = torch.ones(1, 5) * 0.2
pos_embeds32[:, 0] = -0.2

total_tokens = sum(num_tokens)
downsized_img = test_tensor[:, ::2, ::2]

patches32 = downsized_img.reshape(3, -1)
patches16 = test_tensor.reshape(3, -1)

all_patches = torch.cat([patches16, patches32], dim=1)
mask16 = mask16.flatten(1)
mask32 = mask32.flatten(1)
combined_mask = torch.cat([mask16, mask32], dim=1).bool()
selected_patches = all_patches[combined_mask]

pos_embeds_to_add = torch.cat([pos_embeds[:, 1:], pos_embeds32[:, 1:]], dim=1)
pos_embeds_to_add = pos_embeds_to_add.repeat(B, 1)
selected_pos_embeds = pos_embeds_to_add[combined_mask]

#selected_patches = selected_patches + selected_pos_embeds
#ls_token = cls_token + pos_embeds[:, :1]


expanded_output = torch.zeros((total_tokens)).float()
output_mask = torch.ones(total_tokens)
expanded_output[cls_token_indices] = cls_token
output_mask[cls_token_indices] = 0
expanded_output[output_mask.bool()] = selected_patches.float()
expanded_output

Cls token indices:  [0, 14, 25]


tensor([420.,   0.,   1.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
         12.,   2., 420.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
         22., 420.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
         33.,  34.,  35.,  36.,  37.,  38.])

In [22]:
expanded_output[cls_token_indices]

tensor([419.9000, 419.9000, 419.9000])

In [ ]:
def forward(self, x, entropy_maps):
        B = x.shape[0]
        # Set this explicitly. Will make formatting a bit easier.
        # Only return the patches we will actually use.
        with torch.no_grad():
            downsized_img = F.interpolate(x, scale_factor=0.5, mode='bilinear', align_corners=False)
            mask32, mask16, seqlens, cls_token_indices, total_tokens = self.construct_masks(x, entropy_maps)

            patches32 = downsized_img.reshape(B, -1, 3, 16, 16)
            patches16 = x.reshape(B, -1, 3, 16, 16)
            all_patches = torch.cat([patches16, patches32], dim=1)
            mask16 = mask16.flatten(1)
            mask32 = mask32.flatten(1)
            combined_mask = torch.cat([mask16, mask32], dim=1).bool()
            selected_patches = all_patches[combined_mask]

        patch_embeds = self.patch_embed16(selected_patches).squeeze(1)
        embed_dim = patch_embeds.shape[-1]
        # Add pos embed, skip cls token. 
        pos_embeds_to_add = torch.cat([self.pos_embed[:, 1:], self.pos_embed32[:, 1:]], dim=1)
        # repeat along batch dimension.
        pos_embeds_to_add = pos_embeds_to_add.repeat(B, 1, 1)
        selected_pos_embeds = pos_embeds_to_add[combined_mask]

        patch_embeds = patch_embeds + selected_pos_embeds
        cls_token = self.cls_token + self.pos_embed[:, :1]
        # Add the cls tokens in the right spots, and the embeddings everywhere else.
        # Total tokens already handles the class token, no need to add an extra 1.
        expanded_output = torch.zeros((total_tokens, embed_dim), device=cls_token.device)
        output_mask = torch.ones((total_tokens))
        expanded_output[cls_token_indices] = cls_token
        output_mask[cls_token_indices] = 0
        expanded_output[output_mask.bool()] = patch_embeds
        # Add a batch dim = 1.
        expanded_output = expanded_output.unsqueeze(0)
        
        return expanded_output, seqlens, cls_token_indices